# Introduction


La classification des sons est l'une des applications les plus utilisées dans **Audio Deep Learning**. Cela implique d'apprendre à classer les sons et à prédire la catégorie de ce son. Ce type de problème peut être appliqué à de nombreux scénarios pratiques, par exemple la classification de clips musicaux pour identifier le genre de la musique, ou la classification de courts énoncés par un ensemble de locuteurs pour identifier le locuteur en fonction de la voix.

Tout comme la classification des chiffres écrits à la main à l'aide de l'ensemble de données MNIST considérée comme un problème de type "Hello World" pour introduire la vision par ordinateur, la classification audio est lui aussi considérée comme le problème d'introduction pour Audio Deep Learning.

Dans cet article, nous allons parcourir une application de démonstration simple afin de comprendre l'approche utilisée pour résoudre ces problèmes de classification audio. Mon objectif tout au long de ce tutoriel sera de te faire comprendre non seulement comment ces programmes fonctionnent, mais aussi, pourquoi ils fonctionnent de cette façon.

# Les données

Il existe de nombreux ensembles de données appropriés disponibles pour les sons de différents types. Ces ensembles de données contiennent un grand nombre d'échantillons audio, ainsi qu'une étiquette de classe pour chaque échantillon qui identifie le type de son dont il s'agit, en fonction du problème qu'on essaie de résoudre. Ces étiquettes de classe peuvent souvent être obtenues à partir d'une partie du nom de fichier de l'échantillon audio ou du nom du sous-dossier dans lequel se trouve le fichier. Alternativement, les étiquettes de classe sont spécifiées dans un fichier de métadonnées séparé, généralement au format TXT, JSON ou CSV.

Pour notre exemple de projet, nous allons utiliser l'ensemble de données [**Urban Sound 8K**](https://urbansounddataset.weebly.com/urbansound8k.html) qui consiste en un corpus de sons ordinaires enregistrés qu'on retrouve dans notre vie quotidienne en ville. Les sons sont rangés en 10 classes telles que le forage, les aboiements de chiens et les sirènes. Chaque échantillon sonore est étiqueté avec la classe à laquelle il appartient.

Après avoir téléchargé ce jeu de données, on remarque qu'il se compose de deux parties :

- Les **fichiers audio** dans le dossier `audio` qui comporte 10 sous-dossiers nommés de `fold1` à `fold10`. Dans chaque sous-dossier on trouve un certain nombre d'échantillons audio avec l'extension **.wav**, par exemple : `pli1/103074–7–1–0.wav`. Chaque échantillons durent environ 4 secondes.
- Les **métadonnées** dans le dossier `metadata` qui contient un fichier nommé `UrbanSound8K.csv` qui contient des informations sur chaque échantillon audio du jeu de données, telles que son nom de fichier, son étiquette de classe, l'emplacement du sous-dossier 'fold', etc. L'étiquette de classe est un entier qui vari de 0 à 9 pour chacune des 10 classes, par exemple : le chiffre 0 désigne climatiseur, 1 : un klaxon de voiture, etc.

Comme pour la plupart des problèmes de deep learning, nous suivrons les étapes présentées dans la figure suivante :

![Flux de travail d'apprentissage en profondeur](images/dl_steps.png)

Pour notre tâche de classification, les caractéritiques $X$ désigneront les chemins vers des fichiers audio et les étiquettes cibles $y$ vont désignés les noms de classe.

## Récupération des données

Pour télécharger l'ensemble de données, on va juste exécuter la commande suivante : 
```sh
wget -c https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
```
Voici le code à exécuter dans un noteboock :

In [ ]:
!wget -c https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz

## Préparation des données d'entraînement

De nombreux ensembles de données se composent uniquement de fichiers audio organisés dans une structure de dossiers à partir de laquelle on peut extraire les étiquettes de classe. Pour préparer nos données d'entraînement dans le format necessaire pour notre modèle, nous procédera en trois (03) étapes :

- Analyser le répertoire pour préparez une liste de tous les chemins de fichiers audio;
- Extraire l'étiquette de classe de chaque fichier audio du nom de son dossier parent;
- Faire correspondre chaque nom de classe à un numéro unique.

Étant donné que l'ensemble de données contient déjà un fichier CSV de métadonnées contenant les informations recherchées sur chaque fichier audio, on va l'utiliser directement.

In [4]:
import pandas as pd


# On ouvre et on affiche les 5 premiers lignes du fichier.
metadata_df = pd.read_csv("UrbanSound8K.csv")
metadata_df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


On a besoin d'une créer une colonnes de plus pour préparer les données à d'entraînement. Il s'agit de la colonne qui contiendra le chemin d'accès vers chaque fichier audio. Pour cela, on va ajouter une nouvelle colonne nommée `'relative_path'` qui représentera le chemin relatif vers chaque fichier audio.

In [7]:
# On construit le chemin relatif vers chaque fichier audio.
metadata_df['relative_path'] = '/fold' + metadata_df['fold'].astype(str) \
        + '/' \
        + metadata_df['slice_file_name'].astype(str) \

# On affiche un apperçu.
metadata_df[['relative_path']].head()

,relative_path
0,/fold5/100032-3-0-0.wav
1,/fold5/100263-2-0-117.wav
2,/fold5/100263-2-0-121.wav
3,/fold5/100263-2-0-126.wav
4,/fold5/100263-2-0-137.wav


On a juste besoin de deux colonnes dans cette table de données pour constituer la dataset d'entraînement, donc récupérons les dans une nouvelle variable.

In [9]:
# On sélectionne donc les colonnes 'relative_path' et `'classID'`
dataset_df = metadata_df[['relative_path', 'classID']]
dataset_df.head()

,relative_path,classID
0,/fold5/100032-3-0-0.wav,3
1,/fold5/100263-2-0-117.wav,2
2,/fold5/100263-2-0-121.wav,2
3,/fold5/100263-2-0-126.wav,2
4,/fold5/100263-2-0-137.wav,2


Le faite d'avoir le chemin d'accès vers les fichiers audio va nous faciliter les choses pour préparation des données d'apprentissage.

En effet, on ne peut pas faire entrer des chemins de fichiers audio directement dans le modèle. Nous devons donc lire et charger les données audio de chaque fichier et les traiter afin qu'elles soient dans un format exploitable par le modèle.

Ce prétraitement audio sera entièrement effectué dynamiquement au moment de l'exécution de programme d'entraînement. Cette approche est similaire à celle des fichiers images en vision par ordinateur. Car, comme les données d'image, les données audio sont assez volumineuses et gourmandes en mémoire, ce qui fait qu'on ne peut pas charger l'intégralité de l'ensemble de données en mémoire à l'avance. Donc, les seules informations qu'on arriver à conserver en mémoire sont les chemins d'accès vers ces fichiers audio (ou image).

Les données audio sont traitées en appliquant une série de transformations. Exactement comme pour les images, nous pourrions un pipeline de transformations constitué d'une liste de fonctions de traitement audio.

Voici la liste des fonctions à programmer :

1. Lecture de fichier audio;
2. Conversion en 2 canaux (stereo);
3. Normalisation du taux d'échantillonnage;
4. Redimensionnement à la même longueur (durée);
5. Décalage temporel pour l'augmentation;
6. Génération de Spectrogramme Mel;
7. Masquage temporel et fréquentiel pour l'augmentation.


### Lecture de fichier audio

La première chose dont nous avons besoin est de lire et de charger le fichier audio au format **.wav**. Puisque nous utilisons [**Pytorch**](https://pytorch.org/) pour cet exemple, l'implémentation ci-dessous utilise `torchaudio` pour le traitement audio, mais le package python `librosa` peut aussi faire l'affaire.

In [14]:
from typing import Tuple
import torch
import torchaudio
from torchaudio import transforms


def read(audio_file: str) -> Tuple[torch.Tensor, int]:
    """ Fonction de lecture des données audio.

    On utilise la fonction `load()` de `torchaudio`.
    Pour plus d'information, voici un lien :
    https://pytorch.org/audio/stable/tutorials/audio_io_tutorial.html#loading-audio-data

    Arguments:
        audio_file (`str`): Le chemin d'accès vers le fichier audio.

    Returns:
        `tuple`: constitué d'un tenseur et d'un entier.
           * Le tenseur nommé `waveform` représente les données audio;
           * et l'entier nommé `sample_rate` représente le taux
             d'échantillonnage.

    Example:
        >>> waveform, sample_rate = read('./pathto/audio_file.wav')
        >>> print(sample_rate)
        44100
    """
    waveform, sample_rate = torchaudio.load(audio_file)
    return (waveform, sample_rate)

### Conversion en 2 canaux (stereo)

Dans un ensemble de données de fichiers audio, il peut arriver qu'on ait certains fichiers son sont **mono** (c'est-à-dire 1 canal audio) tandis que la plupart d'entre eux soient **stéréo** (c'est-à-dire 2 canaux audio). Étant donné que notre modèle s'attend à ce que tous les données d'entrées aient les mêmes dimensions, alors nous sommes amenés à convertir les fichiers audio mono en stéréo, en dupliquant juste le premier canal sur le second.

In [15]:
from typing import Tuple
import torch


def rechannel(audio: Tuple[torch.Tensor, int],
              channel_count: int) -> Tuple[torch.Tensor, int]:
    """ Fonction de conversion de canal.

    La conversion de mono en stéréo revient juste
    à concatener une copie du tableau de données
    audio (waveform) à lui-même.

    Arguments:
        audio (:obj:`tuple`): Le tuple waveform et sample_rate.
        channel_count (`int`): Le nombre de cannaux souhaités.

    Returns:
        `tuple`: Le couple formé du tenseur modifié
                 et du taux d'échantillonnage.

    Example:
        >>> audio = read('./pathto/audio_file.wav')
        >>> audio_mono = rechannel(audio, 1)  # converti en mono.
        >>> audio_stereo = rechannel(audio, 2)  # converti en stéréo.
        >>>
    """
    if channel_count not in (1, 2):
        raise ValueError("Le nombre de channal doit être 1 ou 2.")

    wave_form, sample_rate = audio
    # NOTE: Le nombre de cannal est égal
    #       au nombre de lignes du tableau wave_form.
    if wave_form.shape[0] == channel_count:
        # Plus la peine de continuer si le nombre de cannaux de ce audio
        # est déjà le nombre de cannal souhaité.
        return audio

    wave_form_rech = None
    if channel_count == 2:
        # on convertie en stéréo:
        wave_form_rech = torch.cat([wave_form, wave_form])
    elif channel_count == 1:
        # on convertie en mono:
        wave_form_rech = wave_form[:1, :]

    return (wave_form_rech, sample_rate)


### Normalisation du taux d'échantillonnage

Toujours dans cette même ensemble de données, certains fichiers audio sont échantionnés à une fréquence de 48000 Hz, alors que la plupart sont échantillonnés à une fréquence de 44100 Hz.

Pour rappel, en mécanique, la fréquence est le nombre de tours enrégistrés en une seconde et un tour correspond une période du signal. En traitement numérique du signal, les signaux audio sont généralement échantillonnés à une fréquence de **44100 Hz**, ce qui signifie les données audio récupérées en une seconde sont stockées dans un **tableau de taille 44100**.

Donc, pour certains fichiers audio de notre dataset, une seconde d'audio aura une taille de tableau de 48 000, or qu'elle devrait avoir une taille de tableau plus petite, 44 100 comme la majorité. Ce sont ces raisons qui nous amène à vouloir normaliser en convertissant tout l'audio au même taux d'échantillonnage afin que tous les tableaux aient les mêmes dimensions.

In [17]:
from typing import Tuple
import torch
from torchaudio import transforms as T


def resample(audio: Tuple[torch.Tensor, int],
             new_sample_rate: int) -> Tuple[torch.Tensor, int]:
    """ Fonction de normalisation de la fréquence d'échantionnage.

    On va utiliser la classe `Resample` du module `transforms`
    du package `torchaudio` pour effectuer cette normalisation.

    Arguments:
        audio (:obj:`tuple`): Le tuple waveform et sample_rate.
        new_sample_rate (`int`): La nouvelle fréquence d'échantillonnage
                                 vers laquelle on veut convertir les données
                                 audio.

    Returns:
        `tuple`: Le couple formé du tenseur modifié
                 et du taux d'échantillonnage.

    Example:
        >>> audio = read('./pathto/audio_file.wav')
        >>> audio = resample(audio, 44100)
        >>>
    """
    wave_form, sample_rate = audio
    if sample_rate == new_sample_rate:
        # Il n'y plus rien à faire d'autre.
        return audio

    resample_obj = T.Resample(sample_rate, new_sample_rate)
    wave_form_resample = resample_obj(wave_form[:1, :])

    num_channels = wave_form.shape[0]
    if num_channels > 1:
        # S'il y a un second canal.
        wf_second_channel = resample_obj(wave_form[1:, :])
        wave_form_resample = torch.cat([wave_form_resample, wf_second_channel])

    return (wave_form_resample, new_sample_rate)
